In [1]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import gurobipy_pandas as gppd

# set intercative mode
gppd.set_interactive()

In [2]:
# importing data
hubs = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_hubs.csv")
network = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_network.csv")
plants = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_plants.csv")
roads = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_roads.csv")
suppliers = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_suppliers.csv")
railroads = pd.read_csv(r"D:\1. UTK PhD\Fall 23\IE 522\Final Project\OneDrive_1_12-5-2023\TX_railroads.csv")

In [3]:
hubs.set_index('hub', inplace=True)
hubs.head()

,index,latitude,longitude,invest,capacity
hub,,,,,
17201,1,33.64844,-95.56841,3476219,300000
17218,2,33.64124,-96.60666,3476219,300000
17359,3,29.31990,-96.10283,3476219,300000
17372,4,30.75623,-98.67770,3476219,300000
17395,5,30.84636,-96.98711,3476219,300000


In [4]:
network.head()

,counties,hubs,plants,techs,demand
0,254,33,167,1,1476310602


In [5]:
plants.set_index('plant', inplace=True)
plants.head()

,index,latitude,longitude,tech,invest,capacity,yield
plant,,,,,,,
541,1,30.220353,-97.740436,1,130956797,152063705,232
542,2,29.676724,-98.635234,1,130956797,152063705,232
543,3,33.760315,-96.559867,1,130956797,152063705,232
544,4,29.531920,-98.286901,1,130956797,152063705,232
545,5,29.688568,-98.562424,1,130956797,152063705,232


In [6]:
roads.set_index(['county','hub'], inplace=True)
roads.head()

index  distance       cost
county hub                              
48001  17201      1  219.6090  40.580691
       17218      2  264.7658  47.661277
       17359      3  370.6750  64.267840
       17372      4  420.0082  72.003286
       17395      5  199.2915  37.394907

In [7]:
# cleaning dataset by removing NA values
suppliers.dropna(inplace=True)
suppliers.county = suppliers.county.astype(int)
suppliers.set_index('county',inplace=True)
suppliers.head()

,index,supply
county,,
48001,1.0,13131.971710
48003,2.0,1177.351950
48005,3.0,3854.618542
48007,4.0,308.182629
48009,5.0,19802.136510


In [8]:
railroads.set_index(['hub','plant'], inplace=True)
railroads.head()

index     distance       cost  loading  capacity
hub   plant                                                  
17201 541        1   922.023251  17.095711  3066792    338000
      542        2  1160.842967  20.923752  3066792    338000
      543        3   186.924258   5.312809  3066792    338000
      544        4  1074.699073  19.542951  3066792    338000
      545        5  1160.842967  20.923752  3066792    338000

In [9]:
# create gurobi model
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-17


In [10]:
### Variables
yhub = gppd.add_vars(m, hubs, name='yhub', vtype=GRB.BINARY)
yplant = gppd.add_vars(m, plants, name='yplant', vtype=GRB.BINARY)
ytruck = gppd.add_vars(m, roads, name='ytruck', vtype=GRB.BINARY)
ytrain = gppd.add_vars(m, railroads, name='ytrain', vtype=GRB.BINARY)

c_to_h = gppd.add_vars(m, roads, name='c_to_h', vtype=GRB.CONTINUOUS, lb=0) # Flow from County to hub
h_to_p = gppd.add_vars(m, railroads, name='h_to_p', vtype=GRB.CONTINUOUS, lb=0) # Flow from hub to plant

In [11]:
### parameters
"""
demand = network['demand]
yield = plants['yield']
cplant = plants['invest']
chub = hubs['invest']
"""
# fuel from external source

"\ndemand = network['demand]\nyield = plants['yield']\ncplant = plants['invest']\nchub = hubs['invest']\n"

In [12]:
### Objective
m.setObjective((plants['invest'] * yplant).sum() + (hubs['invest'] * yhub).sum() + (ytruck * c_to_h * roads['cost']).sum() \
               + (ytrain * railroads['cost']).sum() + (ytrain * h_to_p * railroads['loading']).sum(), 
               GRB.MINIMIZE)

In [13]:
### Constraints
# flow conservation through hub
flow_hub = gppd.add_constrs(m,
                            (yhub * c_to_h).groupby('hub').sum(),
                            GRB.EQUAL,
                            (yhub * h_to_p).groupby('hub').sum(),
                            name='hub_flow'
                            )

In [14]:
# hub capacity
cap_hub = gppd.add_constrs(m,
                           (yhub * c_to_h).groupby('hub').sum(),
                           GRB.LESS_EQUAL,
                           hubs['capacity'],
                           name='hub_capacity'
                           )

In [15]:
# train capacity
cap_train = gppd.add_constrs(m,
                           h_to_p,
                           GRB.LESS_EQUAL,
                           railroads['capacity'],
                           name='train_capacity'
                           )

In [16]:
# supplier (county) capacity
cap_county = gppd.add_constrs(m,
                           (c_to_h * yhub).groupby('county').sum(),
                           GRB.LESS_EQUAL,
                           suppliers['supply'],
                           name='suppliers_supply'
                           )

In [17]:
# plant capacity
cap_plant = gppd.add_constrs(m,
                           (yplant * h_to_p).groupby('plant').sum(),
                           GRB.LESS_EQUAL,
                           plants['capacity'],
                           name='plant_capacity'
                           )

In [18]:
# demand
demand_constraint = gppd.add_constrs(m,
                                     (yplant * plants['yield'] * h_to_p.groupby('plant').sum()).sum(),
                                     GRB.GREATER_EQUAL,
                                     network['demand'],
                                     name='network_demand'
                                     )

In [19]:
# write the model
m.write("supply_chain_biofuel.lp")

In [20]:
m.params.LogToConsole = 1
m.params.MIPgap = 0.05

m.update()
m.optimize()

Set parameter MIPGap to value 0.05
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 5511 rows, 27986 columns and 5511 nonzeros
Model fingerprint: 0xb6fdfa7e
Model has 13893 quadratic objective terms
Model has 488 quadratic constraints
Variable types: 13893 continuous, 14093 integer (14093 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 2e+02]
  Objective range  [2e+00, 1e+08]
  QObjective range [1e+01, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+05, 3e+05]
  QRHS range       [8e-01, 1e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 5511 rows and 13893 columns
Presolve time: 0.17s
Presolved: 44772 rows, 50393 columns, 152823 nonzeros
Presolved model has 